# Data Scraping Wikipedia for Capstone #

Installing Packages and Importing Libraries

In [1]:
!pip install beautifulsoup4
!pip install lxml

In [2]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

Assigning URL and using Beautiful Soup to import data

In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"

In [4]:
page = urllib.request.urlopen(url)

In [5]:
soup = BeautifulSoup(page, "lxml")

Assign a variable with all tables on web page 

In [6]:
all_tables=soup.find_all("table")
#all_tables

Pulling out the table we want

In [7]:
right_table=soup.find('table', class_='wikitable sortable')

Extracting the rows from the table and adding values to different lists. Each list represents a Column on the table.

In [15]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Turning lists into a Dataframe

In [31]:
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighborhood']=C

Removing cells that are not assigned a value (as indicated with a "Not assigned" string in table)

In [32]:
df = df[~df.Borough.str.contains("Not assigned", '\n')]

Grouping the data together and making it easier to read

In [33]:
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].unique().to_frame()

Getting shape of DataFrame

In [19]:
df.shape

(103, 1)

In [20]:
df

,,Neighborhood
Postcode,Borough,
M1B,Scarborough,"[Rouge, Malvern]"
M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
M1E,Scarborough,"[Guildwood\n, Morningside, West Hill]"
M1G,Scarborough,[Woburn]
M1H,Scarborough,[Cedarbrae\n]
...,...,...
M9N,York,[Weston]
M9P,Etobicoke,[Westmount\n]
M9R,Etobicoke,"[Kingsview Village, Martin Grove Gardens\n, Ri..."


In [34]:
df_2 = pd.read_csv('http://cocl.us/Geospatial_data')

In [22]:
df_2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [38]:
df_2.rename(columns = {'Postal Code':'Postcode'}, inplace = True)

In [39]:
df.merge(df_2, how='inner', on='Postcode')

,Postcode,Neighborhood,Latitude,Longitude
0,M1B,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,"[Guildwood\n, Morningside, West Hill]",43.763573,-79.188711
3,M1G,[Woburn],43.770992,-79.216917
4,M1H,[Cedarbrae\n],43.773136,-79.239476
...,...,...,...,...
98,M9N,[Weston],43.706876,-79.518188
99,M9P,[Westmount\n],43.696319,-79.532242
100,M9R,"[Kingsview Village, Martin Grove Gardens\n, Ri...",43.688905,-79.554724
101,M9V,"[Albion Gardens\n, Beaumond Heights, Humbergat...",43.739416,-79.588437
